In [1]:
from preprocess import process_all_images, preprocess
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf

for light_curve, max_light in process_all_images():
    pass

[]